In [22]:
from haystack_integrations.document_stores.elasticsearch import ElasticsearchDocumentStore
from haystack import Pipeline
from haystack.components.embedders import OpenAITextEmbedder
from haystack_integrations.components.retrievers.elasticsearch import ElasticsearchEmbeddingRetriever
from haystack.utils import Secret
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator


from dotenv import load_dotenv
import os



load_dotenv(".env")
open_ai_key = os.environ.get("OPENAI_API_KEY")

document_store = ElasticsearchDocumentStore(hosts = "http://localhost:9200")


retriever = ElasticsearchEmbeddingRetriever(document_store=document_store)
text_embedder = OpenAITextEmbedder(api_key=Secret.from_token(open_ai_key))



template = """
Given the following information, analyze the information and generate a brief summary that addresses the question, 
at the end of your summary, ensure that you list the URLs of the documents that are relevant to the question.

Ensure that you also list the list of stock symbols that are mentioned in the documents.

Context:
Context:
{% for doc in documents %}
    Document: {{ doc.content }} 
    Headline: {{ doc.meta['headline'] }}  
    Summary {{doc.meta['summary']}} 
    Stock symbols {{doc.meta['symbols']}}
    URLs {{doc.meta['url']}}
    \n
{% endfor %};

Question: {{question}}
Answer:
"""

prompt_builder = PromptBuilder(template=template)
generator = OpenAIGenerator(api_key=Secret.from_token(open_ai_key), model="gpt-3.5-turbo")




In [23]:
query_pipeline = Pipeline()
query_pipeline.add_component("text_embedder", text_embedder)
query_pipeline.add_component("retriever", retriever)
query_pipeline.add_component("prompt_builder", prompt_builder)
query_pipeline.add_component("llm", generator)

query_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
query_pipeline.connect("retriever", "prompt_builder.documents")
query_pipeline.connect("prompt_builder", "llm")




🚅 Components
  - text_embedder: OpenAITextEmbedder
  - retriever: ElasticsearchEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [27]:
from IPython.display import display, Markdown, Latex

result = query_pipeline.run({"text_embedder": {"text": "How was stock impacted by the actions of Elon Musk"}})


display(Markdown(result['llm']['replies'][0]))


In recent news, Elon Musk, the CEO of Tesla Inc., has been involved in a series of events related to his compensation package and his relationship with President Joe Biden. Musk responded to claims made by the California Public Employees’ Retirement System CEO regarding his compensation package, expressing disagreement with the decision to vote against it. This development follows a $56 billion pay package that was overturned by a Delaware court, causing volatility in Tesla's stock and the need for a shareholder vote to reinstate it. Additionally, Musk criticized President Biden for prioritizing Tesla's unionization over its environmental impact, leading to tension between the two. Despite not endorsing Donald Trump in the 2024 election, Musk is considering an advisory role for Trump if he is re-elected. This relationship between Musk and Trump signals a potential pivot in their dynamics. On the financial front, Nvidia's stock surged following a funding announcement from Musk's xAI startup. The company saw a significant increase in its stock price, reflecting the positive impact of the funding news. Soft treasury auctions have influenced the stock market, along with a drop in Bitcoin related to Mt. Gox. Lastly, various consumer discretionary stocks experienced both gains and losses in the after-market session, showcasing market volatility.

Stock Symbols: ['TSLA', 'NVDA', 'BTCUSD', 'AEO', 'ATER', 'CLEU', 'FFIE', 'GDHG', 'GSUN', 'LOT', 'RRGB', 'SPGC', 'TCS', 'TRNR', 'ZAPP']

URLs:
1. Elon Musk Fires Back At CalPERS CEO Over Compensation Package Vote: 'What She's Saying Makes No Sense' - https://www.benzinga.com/markets/equities/24/05/39065282/elon-musk-fires-back-at-calpers-ceo-over-compensation-package-vote-what-shes-saying-makes-no-sen
2. Elon Musk Slams Joe Biden, Says President 'Cares A Lot More About Whether Tesla Is Unionized' Over EV Maker's Environmental Impact - https://www.benzinga.com/markets/equities/24/05/39063828/elon-musk-slams-joe-biden-says-president-cares-a-lot-more-about-whether-tesla-is-unionized-over-
3. Bitcoin Spot ETFs Hit Record $14.9B Year-To-Date Inflows On Tuesday; New Poll Shows Biden 'Vulnerable' In 2024 Presidential Race - Top Headlines Today While US Slept - https://www.benzinga.com/news/24/05/39064071/bitcoin-spot-etfs-hit-record-14-9b-year-to-date-inflows-on-tuesday-new-poll-shows-biden-vulnerable-i
4. Nvidia's Stock Jumps 7% Past $1,100 With A Boost From Elon Musk As AI Technology Continues To Drive Market Trends - https://www.benzinga.com/24/05/39072255/nvidias-stock-jumps-7-past-1-100-with-a-boost-from-elon-musk-as-ai-technology-continues-to-drive-mar
5. Elon Musk In The White House? Tesla CEO Could Land Advisory Role For Donald Trump: Report - https://www.benzinga.com/general/politics/24/05/39070289/elon-musk-in-the-white-house-tesla-ceo-could-land-advisory-role-for-donald-trump-report
6. Soft Treasury Auctions Are Hitting The Stock Market, Mt. Gox Related Drop In Bitcoin - https://www.benzinga.com/markets/24/05/39065918/soft-treasury-auctions-are-hitting-the-stock-market-mt-gox-related-drop-in-bitcoin
7. 12 Consumer Discretionary Stocks Moving In Wednesday's After-Market Session - https://www.benzinga.com/insights/movers/24/05/39074036/12-consumer-discretionary-stocks-moving-in-wednesdays-after-market-session